## 增

In [ ]:
INSERT INTO 目标表 （目标列，可选，逗号隔开）
VALUES (目标值，逗号隔开)

In [ ]:
DELETE FROM 表 
WHERE 行筛选条件
（当然也可用子查询）
（若省略 WHERE 条件语句会删除表中所有记录（和 TRUNCATE 等效？）

In [ ]:
UPDATE 表 
SET 要修改的字段 = 具体值/NULL/DEFAULT/列间数学表达式 （修改多个字段用逗号分隔）
WHERE 行筛选

## 查【重点】

In [ ]:
SELECT DISTINCT */表列名/聚合函数/子查询/视图/存储过程 -- 所有能存储表格类的对象
    FROM 表名/子查询/视图/存储过程 -- 所有能存储表格类的对象
LEFT/RIGHT/INNER JOIN ON/USING AND -- 加AND表示复合连接条件，比如多层数据
WHERE IN/BETWEENT/AND/OR/NOT/LIKE/REGEXP/IS NULL /子查询
GROUP BY 条件1，条件2
HAVING IN/BETWEENT/AND/OR/NOT/LIKE/REGEXP/IS NULL /子查询
ORDER BY 条件1，条件2
LIMIT 

## 表连接

In [ ]:
LEFT/RIGHT/INNER JOIN ON/USING AND -- 加AND表示复合连接条件，比如多层数据

## 聚合函数

In [ ]:
 MAX(invoice_date) AS latest_date,  
    -- SELECT选择的不仅可以是列，也可以是数字、列间表达式、列的聚合函数
    MIN(invoice_total) lowest,
    AVG(invoice_total) average,
    SUM(invoice_total * 1.1) total,
    COUNT(*) total_records,
    COUNT(invoice_total) number_of_invoices, 
    -- 和上一个相等
    COUNT(payment_date) number_of_payments,  
    -- 【聚合函数会忽略空值】，得到的支付数少于发票数
    COUNT(DISTINCT client_id) number_of_distinct_clients
    -- DISTINCT client_id 筛掉了该列的重复值，再COUNT计数，会得到不同顾客数
FROM invoices
WHERE invoice_date > '2019-07-01'  -- 想只统计下半年的结果

## 删除对象

In [ ]:
DROP VIEW/PROCEDURE/FUNCTION/TRIGGER/EVENT  IF EXISTS clients_balance;
-- 若不存在这个对象，直接 DROP 会报错，所以要加上 IF EXISTS 先检测有没有这个对象

## 查看对象

In [ ]:
SHOW TRIGGERS/EVENTS
SHOW TRIGGERS/EVENTS LIKE 'payments%'

## 创建对象

### 创建视图

In [ ]:
CREATE VIEW sales_by_client AS
    SELECT 
        client_id,
        name,
        SUM(invoice_total) AS total_sales
    FROM clients c
    JOIN invoices i USING(client_id)
    GROUP BY client_id, name;
    -- 虽然实际上这里加不加上name都一样

### 创建存储过程

In [ ]:
DELIMITER $$
-- delimiter n. 分隔符

    CREATE PROCEDURE 过程名()  
        BEGIN
            ……;
            ……;
            ……;
        END$$

DELIMITER ;

### 参数

In [ ]:
CREATE PROCEDURE 过程名
(
    参数1 数据类型,
    参数2 数据类型,
    ……
)
BEGIN
……
END

### 带默认值的参数

In [ ]:
IF 条件表达式 THEN
    语句1;
    语句2;
    ……;
[ELSE]（可选）
    语句1;
    语句2;
    ……;
END IF;
-- 别忘了【END IF】

### 参数验证

In [ ]:
IF 错误参数条件表达式 THEN
    SIGNAL SQLSTATE '错误类型'
        [SET MESSAGE_TEXT = '关于错误的补充信息']（可选）

### 声明变量

In [ ]:
CREATE DEFINER=`root`@`localhost` PROCEDURE `get_risk_factor`()
BEGIN
    -- 声明三个本地变量，可设默认值
    DECLARE risk_factor DECIMAL(9, 2) DEFAULT 0;
    DECLARE invoices_total DECIMAL(9, 2);
    DECLARE invoices_count INT;

    -- 用SELECT得到需要的值并用INTO传入invoices_total和invoices_count
    SELECT SUM(invoice_total), COUNT(*)
    INTO invoices_total, invoices_count
    FROM invoices;

    -- 用SET语句给risk_factor计算赋值
    SET risk_factor = invoices_total / invoices_count * 5;

    -- 展示最终结果risk_factor
    SELECT risk_factor;        
END

### 创建函数

In [ ]:
1. 参数设置和body内容之间，有一段确定返回值类型以及函数属性的语句段
RETURNS INTEGER
DETERMINISTIC
READS SQL DATA
MODIFIES SQL DATA
……
2. 最后是返回（RETURN）值而不是查询（SELECT）值
RETURN IFNULL(risk_factor, 0);

In [ ]:
CREATE DEFINER=`root`@`localhost` FUNCTION `get_risk_factor_for_client`
(
    client_id INT
) 
RETURNS INTEGER
-- DETERMINISTIC
READS SQL DATA
-- MODIFIES SQL DATA
BEGIN
    DECLARE risk_factor DECIMAL(9, 2) DEFAULT 0;
    DECLARE invoices_total DECIMAL(9, 2);
    DECLARE invoices_count INT;

    SELECT SUM(invoice_total), COUNT(*)
    INTO invoices_total, invoices_count
    FROM invoices i
    WHERE i.client_id = client_id;
    -- 注意不再是整体risk_factor而是特定顾客的risk_factor

    SET risk_factor = invoices_total / invoices_count * 5;
    RETURN IFNULL(risk_factor, 0);       
END

### 创建触发器

In [ ]:
DELIMITER $$

CREATE TRIGGER payments_after_insert
    AFTER INSERT ON payments
    FOR EACH ROW
BEGIN
    UPDATE invoices
    SET payment_total = payment_total + NEW.amount
    WHERE invoice_id = NEW.invoice_id;
END$$

DELIMITER ;

### 创建事件

In [ ]:
DELIMITER $$

CREATE EVENT yearly_delete_stale_audit_row

-- 设定事件的执行计划：
ON SCHEDULE  
    EVERY 1 YEAR [STARTS '2019-01-01'] [ENDS '2029-01-01']    

-- 主体部分：（注意 DO 关键字）
DO BEGIN
    DELETE FROM payments_audit
    WHERE action_date < NOW() - INTERVAL 1 YEAR;
END$$

DELIMITER ;

### 创建事务【难点】

In [ ]:
USE sql_store;

START TRANSACTION;

INSERT INTO orders (customer_id, order_date, status)
VALUES (1, '2019-01-01', 1);
-- 只需明确声明并插入这三个非自增必须（不可为空）字段

INSERT INTO order_items 
-- 所有字段都是必须的，就不必申明了
VALUES (last_insert_id(), 1, 2, 3);

COMMIT;

四个并发问题：
- Lost Updates 丢失更新：两个事务更新同一行，最后提交的事务将覆盖先前所做的更改
- Dirty Reads 脏读：读取了未提交的数据
- Non-repeating Reads 不可重复读取 （或 Inconsistent Read 不一致读取）：在事务中读取了相同的数据两次，但得到了不同的结果
- Phantom Reads 幻读：在查询中缺失了一行或多行，因为另一个事务正在修改数据而我们没有意识到事务的修改，我们就像遇见了鬼或者幽灵


为了解决这些问题，我们有四个标准的事务隔离等级：
  - Read Uncommitted 读取未提交：无法解决任何一个问题，因为事务间并没有任何隔离，他   们   甚至可以读取彼此未提交的更改
  - Read Committed 读取已提交：给予事务一定的隔离，这样我们只能读取已提交的数据，这防止了Dirty Reads 脏读，但在这个级别下，事务仍可能读取同个内容两次而得到不同的结果，因为另一个事务可能在两次读取之间更新并提交了数据，也就是它不能防止
  - Non-repeating Reads 不可重复读取 （或 Inconsistent Read 不一致读取）
  - Repeatable Read 可重复读取：在这一级别下，我们可以确信不同的读取会返回相同的结果，即便数据在这期间被更改和提交
  - Serializable 序列化：可以防止以上所有问题，这一级别还能防止幻读，如果数据在我们执行过程中改变了，我们的事务会等待以获取最新的数据，但这很明显会给服务器增加负担，因为管理等待的事务需要消耗额外的储存和CPU资源

## JSON对象

In [ ]:
SET JSON对象名 = 
'
{
    "key1": value1,
    "key2": value2,
    ……
}
'

### 增

给1号商品增加一系列属性，有两种方法
法1：
用单引号包裹（注意不能是双引号），里面用 JSON 的标准格式：
双引号包裹键 key（注意不能是单引号）
值 value 可以是数、数组、甚至另一个用 {} 包裹的JSON对象
键值对间用逗号隔开

In [ ]:
UPDATE products
SET properties = '
{
    "dimensions": [1, 2, 3], 
    "weight": 10,
    "manufacturer": {"name": "sony"}
}
'
WHERE product_id = 1;

法2：
也可以用 MySQL 里的一些针对 JSON 的内置函数来创建商品属性：

In [ ]:
UPDATE products
SET properties = JSON_OBJECT(
    'weight', 10,
    -- 注意用函数的话，键值对中间是逗号而非冒号
    'dimensions', JSON_ARRAY(1, 2, 3),
    'manufacturer', JSON_OBJECT('name', 'sony')
)
WHERE product_id = 1;

### 查
法1 :
使用 JSON_EXTRACT(JSON对象, '路径') 函数，其中：
- 第1参数指明 JSON 对象
- 第2参数是用单引号包裹的路径，路径中 $ 表示当前对象，点操作符 . 表示对象的属性

In [ ]:
SELECT product_id, JSON_EXTRACT(properties, '$.weight') AS weight
FROM products
WHERE product_id = 1;

法2  
更简便的方法，使用列路径操作符 -> 和 ->>，后者可以去掉结果外层的引号
用法是：JSON对象 -> '路径'

In [ ]:
SELECT properties -> '$.weight' AS weight
FROM products
WHERE product_id = 1;
-- 结果为：10

SELECT properties -> '$.dimensions' 
……
-- 结果为：[1, 2, 3]

SELECT properties -> '$.dimensions[0]' 
-- 用中括号索引切片，且序号从0开始，与Python同
……
-- 结果为：1

SELECT properties -> '$.manufacturer'
……
-- 结果为：{"name": "sony"}

SELECT properties -> '$.manufacturer.name'
……
-- 结果为："sony"

SELECT properties ->> '$.manufacturer.name'
……
-- 结果为：sony

通过路径操作符来获取 JSON 对象的特定属性不仅可以用在 SELECT 选择语句中，也可以用在 WHERE 筛选语句中，如：  
筛选出制造商名称为 sony 的产品：

In [ ]:
SELECT 
    product_id, 
    properties ->> '$.manufacturer.name' AS manufacturer_name
FROM products
WHERE properties ->/->> '$.manufacturer.name' = 'sony'

结果为：

| product_id | manufacturer_name|
|1|sony|
Mosh说最后这个查询的 WHERE 条件语句里用路径获取制作商名字时必须用双箭头 ->> 才能去掉结果的双引号，才能使得比较运算成立并最终查找出符合条件的1号产品，但实验发现用单箭头 -> 也可以，但另一方面在 SELECT 选择语句中用单双箭头确实会使得显示的结果带或不带双引号，所以综合来看，单双箭头应该是只影响路径结果 "sony" 是否【显示】外层的引号，但不会改变其实质，所以不会影响其比较运算结果，即单双箭头得出的sony都是 = 'sony' 的

### 改

In [ ]:
USE sql_store;
UPDATE products
SET properties = JSON_SET(
    properties,
    '$.weight', 20,  -- 修改weight属性
    '$.age', 10  -- 增加age属性
)
WHERE product_id = 1;

注意 JSON_SET() 是选择已有的 JSON 对象并修改部分属性然后返回修改后新的 JSON 对象，所以其第1参数是要修改的 JSON 对象，并且可以用

In [ ]:
SET porperties = JSON_SET(properties, ……)

的语法结构来实现对 properties 的修改

### 删

In [ ]:
USE sql_store;
UPDATE products
SET properties = JSON_REMOVE(
    properties,
    '$.weight',
    '$.age'
)
WHERE product_id = 1;

小结
感觉JSON对象就是个储存键值对的字典，可以嵌套，标准格式为：{"key":value,……}
增：利用标准格式或利用 JSON_OBJECT, JSON_ARRAY 等函数
查：JSON_EXTRACT 或 ->/-->，注意表达路径时单引号、 $ 和 . 的使用
改：JSON_SET，注意其原理
删：JSON_REMOVE，原理同上